In [1]:
from utilities import *
from env_bruteforce import *
from gis_handler import GISHandler

In [2]:
all_vars = ['x_wec','x_type_wec','x_pen','p_pen','x_env','p_env','p_wec','p_fish_salmon','pos_env', 'gis_handler', 'p_vessel']

conditions = {'current [m/s]': 'data/Surface Currents m-s (NODP 2016).tif',
              'oxygen [mg/l]': 'data/Surface Oxygen mg-l (NCEI 2019).tif',
              'salinity [PSU]': 'data/Surface Salinity PSU (NCEI 2019).tif',
              'temperature [C]': 'data/Surface Temperature C (NODP 2016).tif',
              'period [s]': 'data/Wave Energy Period s (NREL 2011).tif',
              'height [m]': 'data/Significant Wave Height m (NREL 2011).tif',
              'bathymetry [m]': 'data/Bathymetry Downsampled m (NGDC 1990).tif',
              'distance to port [m]': 'data/Distance to Port m (OCM 2019).tif'}

# high fishing is above average, very high is more than one standard deviation above average
conflicts = {'very high fishing traffic': 'data/Very High Fishing Vessel Traffic (NODP 2022).geojson',
#            'high fishing traffic': 'data/High Fishing Vessel Traffic (NODP 2022).geojson',
             'marine protected areas': 'data/Marine Protected Areas (NMPAC 2020).geojson',
             'danger zones': 'data/Danger Zones and Restricted Areas (OCM 2022).geojson',
             'submarine': 'data/Submarine Transit Lanes (NODP 2016).geojson',
             'torpex': 'data/Cape Cod TORPEX (NODP 2016).geojson',
             'block island': 'data/Block Island Renewable Energy Zone (NODP 2010).geojson',
             'ma wind': 'data/MA Wind Energy Areas (NODP 2015).geojson',
             'wind lease': 'data/Potential Wind Lease Areas (BOEM 2023).geojson',
             'wind planning': 'data/Wind Planning Areas (BOEM 2023).geojson',
             'shipping': 'data/Shipping Lanes (OCS 2015).geojson'}

waters = 'data/Northeast State and Federal Waters (OCM 2018).geojson'

handler = GISHandler(conditions, conflicts, waters)

In [3]:
args = {}
args['handler'] = handler
args['grid_resolution'] = 0.1

best_x_total, aqua_obj_best_total, p_best, best_x_without_conflict, aqua_obj_best_without_conflict = env_bruteforce(all_vars, args)

Manually replaced [(38.45, 45.19), (-75.78, -65.7)] with new bounds of [(38.4, 45.2), (-75.8, -65.7)] to match previously calculated points.


/Users/gabriel/Documents/SEA Lab/aquaculture/gis_handler.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.points = pd.concat([self.points, pd.DataFrame([conditions])], ignore_index=True)
/Users/gabriel/Documents/SEA Lab/aquaculture/env_bruteforce.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.points = pd.concat([self.points, pd.DataFrame([data])], ignore_index=True)
/Users/gabriel/Documents/SEA Lab/aquaculture/env_bruteforce.py:40: FutureWarning: The behavior of DataFrame con

In [5]:
bruteforce_result("without conflict consideration", best_x_total, aqua_obj_best_total, p_best)
bruteforce_result("with conflict consideration", best_x_without_conflict, aqua_obj_best_without_conflict, p_best)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
brute force without conflict consideration
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
design variable:
   pos_lat       43.700 m
   pos_long     -68.900 m
----------------------------------------
 objective function terms:
   Objective_func 30044429.965
   fish_yield     675933.294 [kg]
----------------------------------------
 wave energy converter:
   wec number          5.000 [-]
   wave power          3.717 [kW/m]
   wec_P_ave           8.821 [kW]
   wec AEP        386633.844 [kWh]
----------------------------------------
 price break down:
   wec NPV              29452107.101 [$]
   pen price            5301437.603 [$]
   fish feed price      1765378.722 [$]
   vessel travel NPV    592322.863 [$]
----------------------------------------
 location validation:
   conditions           5475    True
Name: ok-conditions, dtype: object
   scope                5475    Tru